In [1]:
import sys
sys.path.insert(0, '/home/anish/ASC_ML')

In [2]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()

In [3]:
from ASC_ML import dataframe_extractor as de
from ASC_ML import model_generation as model_gen
from ASC_ML import model_gen_train_test as testing
from dask_ml.preprocessing import LabelEncoder
# from dask_ml.preprocessing import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

2022-06-18 17:03:36.432879: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [4]:
from ASC_ML import multiple_model_gen_v1 as multiple

In [5]:
# import dask.dataframe as dd
# import pandas as pd
# data = dd.from_pandas(pd.Series(['a', 'a', 'b'], dtype='category'),
#                      npartitions=1)
# le = LabelEncoder()
# le.fit_transform(data).compute()
# print(type(le.fit_transform(data).compute()))

In [6]:
#For House Prediction Dataset https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data?select=train.csv
directory1 = "/home/anish/ASC-ML-EXP-DATASETS/LinReg-tabular/house-prices-advanced-regression-techniques/train.csv"


# returns list of dask dataframe [singular_df_x, singular_df_y] or [train_df_x, train_df_y, test_df_x, test_df_y]
dataset_list = de.DataframeExtractor_csv(directory1, label_names = ["SalePrice"]).df_list

print("Train Dataset X Columns")
dataset_list[0].head()

Reading single csv from /home/anish/ASC-ML-EXP-DATASETS/LinReg-tabular/house-prices-advanced-regression-techniques/train.csv
Train Dataset X Columns


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1.0,60.0,RL,65.0,8450.0,Pave,NaN,Reg,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,2.0,2008.0,WD,Normal
1,2.0,20.0,RL,80.0,9600.0,Pave,NaN,Reg,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,5.0,2007.0,WD,Normal
2,3.0,60.0,RL,68.0,11250.0,Pave,NaN,IR1,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,9.0,2008.0,WD,Normal
3,4.0,70.0,RL,60.0,9550.0,Pave,NaN,IR1,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,2.0,2006.0,WD,Abnorml
4,5.0,60.0,RL,84.0,14260.0,Pave,NaN,IR1,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,12.0,2008.0,WD,Normal


In [7]:
dataset_list[1].head()

,SalePrice
0,208500.0
1,181500.0
2,223500.0
3,140000.0
4,250000.0


In [8]:
train_x = dataset_list[0].copy()

In [9]:
train_x = train_x.fillna(0)

In [10]:
train_x.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1.0,60.0,RL,65.0,8450.0,Pave,0,Reg,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2.0,2008.0,WD,Normal
1,2.0,20.0,RL,80.0,9600.0,Pave,0,Reg,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,5.0,2007.0,WD,Normal
2,3.0,60.0,RL,68.0,11250.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,9.0,2008.0,WD,Normal
3,4.0,70.0,RL,60.0,9550.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2.0,2006.0,WD,Abnorml
4,5.0,60.0,RL,84.0,14260.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,12.0,2008.0,WD,Normal


In [11]:
train_x['MSSubClass'] = train_x['MSSubClass'].apply(str)

train_x['OverallCond'] = train_x['OverallCond'].astype(str)

train_x['YrSold'] = train_x['YrSold'].astype(str)
train_x['MoSold'] = train_x['MoSold'].astype(str)
train_x.head()

/home/anish/anaconda3/envs/tfgpu/lib/python3.9/site-packages/dask/dataframe/core.py:3946: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('MSSubClass', 'object'))

  warnings.warn(meta_warning(meta))


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1.0,60.0,RL,65.0,8450.0,Pave,0,Reg,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2.0,2008.0,WD,Normal
1,2.0,20.0,RL,80.0,9600.0,Pave,0,Reg,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,5.0,2007.0,WD,Normal
2,3.0,60.0,RL,68.0,11250.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,9.0,2008.0,WD,Normal
3,4.0,70.0,RL,60.0,9550.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2.0,2006.0,WD,Abnorml
4,5.0,60.0,RL,84.0,14260.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,12.0,2008.0,WD,Normal


In [12]:
# a = train_x['LotShape']

In [13]:
# le = LabelEncoder() 
# le.fit_transform(a).compute()
# train_x['LotShape'] = le.fit_transform(train_x['LotShape'])

In [14]:
# train_x.head()

In [15]:
def keyval(allkeys):
    i = 0
    keyvalpairs = list()
    for key in allkeys:
        keyvalpairs.append((key,i))
        i+=1
    return dict(keyvalpairs)

In [16]:
# cols = ['FireplaceQu', 'LotShape']
cols = ['FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold']
keys = {}
for col in cols:
    key = train_x[col].unique()
    keyvalpairs = keyval(key)
    keys.update({col:keyvalpairs})


In [17]:
keys

{'FireplaceQu': {0: 0, 'TA': 1, 'Gd': 2, 'Fa': 3, 'Ex': 4, 'Po': 5},
 'BsmtQual': {'Gd': 0, 'TA': 1, 'Ex': 2, 0: 3, 'Fa': 4},
 'BsmtCond': {'TA': 0, 'Gd': 1, 0: 2, 'Fa': 3, 'Po': 4},
 'GarageQual': {'TA': 0, 'Fa': 1, 'Gd': 2, 0: 3, 'Ex': 4, 'Po': 5},
 'GarageCond': {'TA': 0, 'Fa': 1, 0: 2, 'Gd': 3, 'Po': 4, 'Ex': 5},
 'ExterQual': {'Gd': 0, 'TA': 1, 'Ex': 2, 'Fa': 3},
 'ExterCond': {'TA': 0, 'Gd': 1, 'Fa': 2, 'Po': 3, 'Ex': 4},
 'HeatingQC': {'Ex': 0, 'Gd': 1, 'TA': 2, 'Fa': 3, 'Po': 4},
 'PoolQC': {0: 0, 'Ex': 1, 'Fa': 2, 'Gd': 3},
 'KitchenQual': {'Gd': 0, 'TA': 1, 'Ex': 2, 'Fa': 3},
 'BsmtFinType1': {'GLQ': 0,
  'ALQ': 1,
  'Unf': 2,
  'Rec': 3,
  'BLQ': 4,
  0: 5,
  'LwQ': 6},
 'BsmtFinType2': {'Unf': 0,
  'BLQ': 1,
  0: 2,
  'ALQ': 3,
  'Rec': 4,
  'LwQ': 5,
  'GLQ': 6},
 'Functional': {'Typ': 0,
  'Min1': 1,
  'Maj1': 2,
  'Min2': 3,
  'Mod': 4,
  'Maj2': 5,
  'Sev': 6},
 'Fence': {0: 0, 'MnPrv': 1, 'GdWo': 2, 'GdPrv': 3, 'MnWw': 4},
 'BsmtExposure': {'No': 0, 'Gd': 1, 'Mn': 2, '

In [18]:
train_X = train_x.replace(keys).copy()

In [19]:
train_X['FireplaceQu'].unique().compute()

0    0
1    1
2    2
3    3
4    4
5    5
Name: FireplaceQu, dtype: int64

In [20]:
train_X.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1.0,0,RL,65.0,8450.0,0,0,0,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,0,0,WD,Normal
1,2.0,1,RL,80.0,9600.0,0,0,0,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,1,1,WD,Normal
2,3.0,0,RL,68.0,11250.0,0,0,1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2,0,WD,Normal
3,4.0,2,RL,60.0,9550.0,0,0,1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,0,2,WD,Abnorml
4,5.0,0,RL,84.0,14260.0,0,0,1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,3,0,WD,Normal


In [21]:
cols_to_remove = []
for col in train_X.columns:
#     print(type(col))
    try:
        a = train_X[col].astype(float).compute()
    except ValueError:
#         print('Couldn\'t covert %s to float' % col)
        cols_to_remove.append(col)
        pass

# keep only the columns in df that do not contain string
train_X = train_X[[col for col in train_X.columns if col not in cols_to_remove]]

In [22]:
print(cols_to_remove)

['MSZoning', 'LandContour', 'Utilities', 'LotConfig', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 'Heating', 'Electrical', 'GarageType', 'MiscFeature', 'SaleType', 'SaleCondition']


In [23]:
train_X = train_X.drop(["Id"], axis = 1)

In [24]:
# tsklearn.preprocessing.MinMaxScalerrain_X.head()

In [25]:
len(train_X.columns)

58

In [26]:
len(dataset_list[1].columns)

1

In [27]:
scaler_x = MinMaxScaler()
scaler_x.fit(train_X)
train_X_scaled = scaler_x.transform(train_X)

scaler_y = MinMaxScaler()
scaler_y.fit(dataset_list[1])
train_Y_scaled = scaler_y.transform(dataset_list[1])

In [28]:
# col = train_X.columns.to_list()
# scaler_d = MinMaxScaler()
# train_X.columns = list(train_X.columns)
# scaled_train_X = scaler_d.fit_transform(train_X[col])

In [29]:
train_X_scaled
# train_Y_scaled

array([[0.        , 0.20766773, 0.0334198 , ..., 0.        , 0.        ,
        0.        ],
       [0.07142857, 0.25559105, 0.03879502, ..., 0.        , 0.09090909,
        0.25      ],
       [0.        , 0.2172524 , 0.04650728, ..., 0.        , 0.18181818,
        0.        ],
       ...,
       [0.14285714, 0.21086262, 0.03618687, ..., 0.16129032, 0.09090909,
        1.        ],
       [0.07142857, 0.2172524 , 0.03934189, ..., 0.        , 0.63636364,
        1.        ],
       [0.07142857, 0.23961661, 0.04037019, ..., 0.        , 1.        ,
        0.        ]])

In [30]:
# TRAIN = multiple.Multiple_Model_Gen(train_X_scaled, train_Y_scaled, 58, 1, None, 5)

In [31]:
# TRAIN

In [32]:
# model = TRAIN.get_models()
# model.summary()

In [33]:
# TRAIN.get_all_models()

In [34]:
m = multiple.Multiple_Model_Gen(train_X_scaled, train_Y_scaled, 50, 128, input_shape = 58, min_no_layers = 3, max_no_layers = 3, model_per_batch = 5)

In [35]:
m.get_all_models()

2022-06-18 17:03:49.286864: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-06-18 17:03:49.287069: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-06-18 17:03:49.287448: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-18 17:03:49.287934: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2060 computeCapability: 7.5
coreClock: 1.695GHz coreCount: 30 deviceMemorySize: 5.79GiB deviceMemoryBandwidth: 312.97GiB/s
2022-06-18 17:03:49.288013: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-06-18 17:03:49.292548: I tensorflow/stream_executor/platform/de

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer_dense_16_16_16 (Inp [(None, 58)]         0                                            
__________________________________________________________________________________________________
input_layer_dense_16_16_32 (Inp [(None, 58)]         0                                            
__________________________________________________________________________________________________
input_layer_dense_16_32_16 (Inp [(None, 58)]         0                                            
__________________________________________________________________________________________________
input_layer_dense_16_32_32 (Inp [(None, 58)]         0                                            
______________________________________________________________________________________________

2022-06-18 17:03:54.947494: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


12/12 [==============================] - 7s 4ms/step - loss: 3.2404 - output_layer_dense_16_16_16_loss: 0.6481 - output_layer_dense_16_16_32_loss: 0.6481 - output_layer_dense_16_32_16_loss: 0.6481 - output_layer_dense_16_32_32_loss: 0.6481 - output_layer_dense_32_16_16_loss: 0.6481 - output_layer_dense_16_16_16_mean_absolute_error: 0.7975 - output_layer_dense_16_16_32_mean_absolute_error: 0.7975 - output_layer_dense_16_32_16_mean_absolute_error: 0.7975 - output_layer_dense_16_32_32_mean_absolute_error: 0.7975 - output_layer_dense_32_16_16_mean_absolute_error: 0.7975
Epoch 2/50
12/12 [==============================] - 0s 3ms/step - loss: 3.2282 - output_layer_dense_16_16_16_loss: 0.6456 - output_layer_dense_16_16_32_loss: 0.6456 - output_layer_dense_16_32_16_loss: 0.6456 - output_layer_dense_16_32_32_loss: 0.6456 - output_layer_dense_32_16_16_loss: 0.6456 - output_layer_dense_16_16_16_mean_absolute_error: 0.7962 - output_layer_dense_16_16_32_mean_absolute_error: 0.7962 - output_layer_de

12/12 [==============================] - 0s 3ms/step - loss: 3.2456 - output_layer_dense_16_16_16_loss: 0.6491 - output_layer_dense_16_16_32_loss: 0.6491 - output_layer_dense_16_32_16_loss: 0.6491 - output_layer_dense_16_32_32_loss: 0.6491 - output_layer_dense_32_16_16_loss: 0.6491 - output_layer_dense_16_16_16_mean_absolute_error: 0.7983 - output_layer_dense_16_16_32_mean_absolute_error: 0.7983 - output_layer_dense_16_32_16_mean_absolute_error: 0.7983 - output_layer_dense_16_32_32_mean_absolute_error: 0.7983 - output_layer_dense_32_16_16_mean_absolute_error: 0.7983
Epoch 16/50
12/12 [==============================] - 0s 3ms/step - loss: 3.2471 - output_layer_dense_16_16_16_loss: 0.6494 - output_layer_dense_16_16_32_loss: 0.6494 - output_layer_dense_16_32_16_loss: 0.6494 - output_layer_dense_16_32_32_loss: 0.6494 - output_layer_dense_32_16_16_loss: 0.6494 - output_layer_dense_16_16_16_mean_absolute_error: 0.7983 - output_layer_dense_16_16_32_mean_absolute_error: 0.7983 - output_layer_d

12/12 [==============================] - 0s 4ms/step - loss: 3.2447 - output_layer_dense_16_16_16_loss: 0.6489 - output_layer_dense_16_16_32_loss: 0.6489 - output_layer_dense_16_32_16_loss: 0.6489 - output_layer_dense_16_32_32_loss: 0.6489 - output_layer_dense_32_16_16_loss: 0.6489 - output_layer_dense_16_16_16_mean_absolute_error: 0.7979 - output_layer_dense_16_16_32_mean_absolute_error: 0.7979 - output_layer_dense_16_32_16_mean_absolute_error: 0.7979 - output_layer_dense_16_32_32_mean_absolute_error: 0.7979 - output_layer_dense_32_16_16_mean_absolute_error: 0.7979
Epoch 30/50
12/12 [==============================] - 0s 3ms/step - loss: 3.2340 - output_layer_dense_16_16_16_loss: 0.6468 - output_layer_dense_16_16_32_loss: 0.6468 - output_layer_dense_16_32_16_loss: 0.6468 - output_layer_dense_16_32_32_loss: 0.6468 - output_layer_dense_32_16_16_loss: 0.6468 - output_layer_dense_16_16_16_mean_absolute_error: 0.7963 - output_layer_dense_16_16_32_mean_absolute_error: 0.7963 - output_layer_d

12/12 [==============================] - 0s 3ms/step - loss: 3.2303 - output_layer_dense_16_16_16_loss: 0.6461 - output_layer_dense_16_16_32_loss: 0.6461 - output_layer_dense_16_32_16_loss: 0.6461 - output_layer_dense_16_32_32_loss: 0.6461 - output_layer_dense_32_16_16_loss: 0.6461 - output_layer_dense_16_16_16_mean_absolute_error: 0.7958 - output_layer_dense_16_16_32_mean_absolute_error: 0.7958 - output_layer_dense_16_32_16_mean_absolute_error: 0.7958 - output_layer_dense_16_32_32_mean_absolute_error: 0.7958 - output_layer_dense_32_16_16_mean_absolute_error: 0.7958
Epoch 44/50
12/12 [==============================] - 0s 3ms/step - loss: 3.2328 - output_layer_dense_16_16_16_loss: 0.6466 - output_layer_dense_16_16_32_loss: 0.6466 - output_layer_dense_16_32_16_loss: 0.6466 - output_layer_dense_16_32_32_loss: 0.6466 - output_layer_dense_32_16_16_loss: 0.6466 - output_layer_dense_16_16_16_mean_absolute_error: 0.7968 - output_layer_dense_16_16_32_mean_absolute_error: 0.7968 - output_layer_d

12/12 [==============================] - 1s 2ms/step - loss: 1.9304 - output_layer_dense_32_16_32_loss: 0.6435 - output_layer_dense_32_32_16_loss: 0.6435 - output_layer_dense_32_32_32_loss: 0.6435 - output_layer_dense_32_16_32_mean_absolute_error: 0.7948 - output_layer_dense_32_32_16_mean_absolute_error: 0.7948 - output_layer_dense_32_32_32_mean_absolute_error: 0.7948
Epoch 2/50
12/12 [==============================] - 0s 2ms/step - loss: 1.9330 - output_layer_dense_32_16_32_loss: 0.6443 - output_layer_dense_32_32_16_loss: 0.6443 - output_layer_dense_32_32_32_loss: 0.6443 - output_layer_dense_32_16_32_mean_absolute_error: 0.7953 - output_layer_dense_32_32_16_mean_absolute_error: 0.7953 - output_layer_dense_32_32_32_mean_absolute_error: 0.7953
Epoch 3/50
12/12 [==============================] - 0s 2ms/step - loss: 1.9416 - output_layer_dense_32_16_32_loss: 0.6472 - output_layer_dense_32_32_16_loss: 0.6472 - output_layer_dense_32_32_32_loss: 0.6472 - output_layer_dense_32_16_32_mean_abso

12/12 [==============================] - 0s 2ms/step - loss: 1.9422 - output_layer_dense_32_16_32_loss: 0.6474 - output_layer_dense_32_32_16_loss: 0.6474 - output_layer_dense_32_32_32_loss: 0.6474 - output_layer_dense_32_16_32_mean_absolute_error: 0.7972 - output_layer_dense_32_32_16_mean_absolute_error: 0.7972 - output_layer_dense_32_32_32_mean_absolute_error: 0.7972
Epoch 23/50
12/12 [==============================] - 0s 2ms/step - loss: 1.9400 - output_layer_dense_32_16_32_loss: 0.6467 - output_layer_dense_32_32_16_loss: 0.6467 - output_layer_dense_32_32_32_loss: 0.6467 - output_layer_dense_32_16_32_mean_absolute_error: 0.7960 - output_layer_dense_32_32_16_mean_absolute_error: 0.7960 - output_layer_dense_32_32_32_mean_absolute_error: 0.7960
Epoch 24/50
12/12 [==============================] - 0s 2ms/step - loss: 1.9392 - output_layer_dense_32_16_32_loss: 0.6464 - output_layer_dense_32_32_16_loss: 0.6464 - output_layer_dense_32_32_32_loss: 0.6464 - output_layer_dense_32_16_32_mean_ab

12/12 [==============================] - 0s 2ms/step - loss: 1.9244 - output_layer_dense_32_16_32_loss: 0.6415 - output_layer_dense_32_32_16_loss: 0.6415 - output_layer_dense_32_32_32_loss: 0.6415 - output_layer_dense_32_16_32_mean_absolute_error: 0.7931 - output_layer_dense_32_32_16_mean_absolute_error: 0.7931 - output_layer_dense_32_32_32_mean_absolute_error: 0.7931
Epoch 44/50
12/12 [==============================] - 0s 2ms/step - loss: 1.9536 - output_layer_dense_32_16_32_loss: 0.6512 - output_layer_dense_32_32_16_loss: 0.6512 - output_layer_dense_32_32_32_loss: 0.6512 - output_layer_dense_32_16_32_mean_absolute_error: 0.7994 - output_layer_dense_32_32_16_mean_absolute_error: 0.7994 - output_layer_dense_32_32_32_mean_absolute_error: 0.7994
Epoch 45/50
12/12 [==============================] - 0s 2ms/step - loss: 1.9511 - output_layer_dense_32_16_32_loss: 0.6504 - output_layer_dense_32_32_16_loss: 0.6504 - output_layer_dense_32_32_32_loss: 0.6504 - output_layer_dense_32_16_32_mean_ab